In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trying_delta")\
  .config("spark.jars.packages", "io.delta:delta-core_2.12:2.2.0")\
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
  .getOrCreate()

24/03/31 16:05:51 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.100.203 instead (on interface wlp5s0)
24/03/31 16:05:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/aadi/miniconda3/envs/pyspark_env/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/aadi/.ivy2/cache
The jars for the packages stored in: /home/aadi/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3beb7ccb-4799-4454-b4bd-4fb2b4f044cc;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.2.0/delta-core_2.12-2.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.2.0!delta-core_2.12.jar (872ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.2.0/delta-storage-2.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;2.2.0!delta-storage.jar (149ms)
:: resolution report :: resolve 993ms :: artifacts dl 1025ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from

24/03/31 16:05:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/31 16:05:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Read and Write a Delta Table

In [8]:
from pandas import DataFrame
df = DataFrame({'a': [1, 2, 3], 'val': [1, 2, 3]})
df.to_csv('./test_csv.csv', index=False)

df = spark.read.csv('./test_csv.csv', header=True)
df.write.save('./test_csv.delta')

In [9]:
# read a delta table
PATH_TO_DELTA = './test_csv.delta/'
delta_table = spark.read.load(PATH_TO_DELTA)

In [23]:
# get location of managed table
delta_table.write.format('delta').saveAsTable('test_csv')
spark.sql('DESCRIBE DETAIL test_csv').show()

24/03/31 16:14:21 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+------+--------------------+----------------+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+
|format|                  id|            name|description|            location|           createdAt|        lastModified|partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|
+------+--------------------+----------------+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+
| delta|b8794435-fdb3-4bb...|default.test_csv|       null|file:/storage/pro...|2024-03-31 16:14:...|2024-03-31 16:14:...|              []|       1|        656|        {}|               1|               2|


## View Delta Table History and Load Previous Versions

In [26]:
spark.sql('DESCRIBE HISTORY default.test_csv').show(vertical=True)

-RECORD 0-----------------------------------
 version             | 0                    
 timestamp           | 2024-03-31 16:14:... 
 userId              | null                 
 userName            | null                 
 operation           | CREATE TABLE AS S... 
 operationParameters | {isManaged -> tru... 
 job                 | null                 
 notebook            | null                 
 clusterId           | null                 
 readVersion         | null                 
 isolationLevel      | Serializable         
 isBlindAppend       | true                 
 operationMetrics    | {numFiles -> 1, n... 
 userMetadata        | null                 
 engineInfo          | Apache-Spark/3.3.... 



In [28]:
df1 = spark.read.option("timestampAsOf", "2019-01-01").table("people10m") # using date
df2 = spark.read.option("versionAsOf", 123).load("/tmp/delta/people10m") # using numbers

+---+---+
|  a|val|
+---+---+
|  1|  1|
|  2|  2|
|  3|  3|
+---+---+



restoring a table 
```sql
RESTORE TABLE db.target_table TO VERSION AS OF <version>
RESTORE TABLE delta.`/data/target/` TO TIMESTAMP AS OF <timestamp>
``````

## Create, Overwrite, Merge and read Feature Store Tables in Machine Learning Workflows

Links: 
https://docs.databricks.com/en/machine-learning/feature-store/workspace-feature-store/feature-tables.html  

### Create Table

In [ ]:
from databricks.feature_store import feature_table

def compute_customer_features(data):
  ''' Feature computation code returns a DataFrame with 'customer_id' as primary key'''
  pass

# create feature table keyed by customer_id
# take schema from DataFrame output by compute_customer_features
from databricks.feature_store import FeatureStoreClient

customer_features_df = compute_customer_features(df)

fs = FeatureStoreClient()

customer_feature_table = fs.create_table(
  name='recommender_system.customer_features',
  primary_keys='customer_id',
  schema=customer_features_df.schema,
  description='Customer features'
)

# An alternative is to use `create_table` and specify the `df` argument.
# This code automatically saves the features to the underlying Delta table.

# customer_feature_table = fs.create_table(
#  ...
#  df=customer_features_df,
#  ...
# )

# To use a composite key, pass all keys in the create_table call

# customer_feature_table = fs.create_table(
#   ...
#   primary_keys=['customer_id', 'date'],
#   ...
# )

# Use write_table to write data to the feature table
# Overwrite mode does a full refresh of the feature table

fs.write_table(
  name='recommender_system.customer_features',
  df = customer_features_df,
  mode = 'overwrite'
)


### Register Existing Table

In [ ]:
fs.register_table(
  delta_table='database.table_name',
  primary_keys=['id'],
  description='something meaningful'
)

### Add New Features

Basically just rerun `write_table` with the new features, this also works to update only specific rows, where the primary key doesn't exist, the rows do not get edited

In [ ]:
from databricks.feature_store import feature_table

def compute_customer_features_2(data):
  ''' Feature computation code returns a DataFrame with 'customer_id' as primary key'''
  pass

# create feature table keyed by customer_id
# take schema from DataFrame output by compute_customer_features
from databricks.feature_store import FeatureStoreClient

customer_features_df = compute_customer_features_2(df)

fs = FeatureStoreClient()

fs.write_table(
  name='recommender_system.customer_features',
  df = customer_features_df,
  mode = 'merge'
)

## Read from a Feature Table

In [ ]:
fs = feature_store.FeatureStoreClient()
customer_features_df = fs.read_table(
  name='recommender.customer_features',
)

## Tags
1. To add tags, use the `tags` parameter of type `dict` when calling `create_table`
2. Use `feature_store_client_instance.set_feature_table_tag` or `.delete_feature_table_tag` for existing tables  

## Add Data Sources 

In [ ]:
from databricks.feature_store import FeatureStoreClient
fs = FeatureStoreClient()

# Use `source_type="table"` to add a table in the metastore as data source.
fs.add_data_sources(feature_table_name="clicks", data_sources="user_info.clicks", source_type="table")

# Use `source_type="path"` to add a data source in path format.
fs.add_data_sources(feature_table_name="user_metrics", data_sources="dbfs:/FileStore/user_metrics.json", source_type="path")

# Use `source_type="custom"` if the source is not a table or a path.
fs.add_data_sources(feature_table_name="user_metrics", data_sources="user_metrics.txt", source_type="custom")


### Deleta a Feature Table

In [ ]:
# when calling this, the underlying delta table is also dropped 

fs.drop_table(
  name='recommender_system.customer_features'
)